<a href="https://colab.research.google.com/github/girotodenis/ReconhecimentoImagemVisaoComputacional/blob/master/Projeto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NOME COMPLETO DOS INTEGRANTES DO GRUPO:
**Davi Miranda Gonçalves**, 
**Denis SIlva Giroto**,
**Naiara Moura Pires**

MATRÍCULAS DOS INTEGRANTES DO GRUPO:
**1931143030**,
**1931143029**,
**1931143033**

In [ ]:
!git clone -l -s https://github.com/girotodenis/ReconhecimentoImagemVisaoComputacional.git projeto
#%cd cloned-repo
!ls

fatal: destination path 'projeto' already exists and is not an empty directory.
projeto  sample_data  yolov3.weights


In [ ]:
!curl -LJO https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov3.weights --output yolov3.weights

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   638  100   638    0     0   3584      0 --:--:-- --:--:-- --:--:--  3584
  0  236M    0 15949    0     0  44058      0  1:33:49 --:--:--  1:33:49 44058
curl: (23) Failed writing body (0 != 15949)


In [ ]:
%ls 'projeto' 

Atividade1.ipynb  gravar_video.ipynb  Projeto3.ipynb  README.md   yolov4.cfg
Atividade2.ipynb  imagens/            Projeto4.ipynb  videos/
coco.names        Projeto2.ipynb      Projeto.ipynb   yolov3.cfg


In [ ]:
from google.colab.patches import cv2_imshow

import numpy as np
import cv2
import imutils

import time 


In [ ]:
net = cv2.dnn.readNet('yolov3.weights','./projeto/yolov3.cfg')

In [ ]:
classes = []
with open('./projeto/coco.names','r')as f:
  classes = [line.strip() for line in f.readlines()]
print(classes)

layer_names = net.getLayerNames()
outputLayers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
print(outputLayers)

cores = np.random.uniform(0,255,size=(len(classes), 3))

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
['yolo_82', 'yolo_94', 'yolo_106']


In [ ]:
def encontrar_objetos_dnn(outs):
  class_ids = []
  confidences = []
  boxes = []

  for out in outs:
    for detection in out:
      scores = detection[5:]
      class_id = np.argmax(scores)
      confidence = scores[class_id]

      if confidence>0.6:
        x,y,w,h,cx,cy = calcula_box(detection,width,height)
        boxes.append([x,y,w,h,cx,cy])
        confidences.append(confidence)
        class_ids.append(class_id)
      
      # print(class_ids)

  return class_ids, confidences, boxes

In [ ]:
def calcula_box(detection, width, height):
  center_x = int(detection[0]*width)
  center_y = int(detection[1]*height)
  w = int(detection[2]*width)
  h = int(detection[3]*height)
  x = int(center_x - w/2)
  y = int(center_y - h/2)
  return [x,y,w,h,center_x,center_y]

In [ ]:
def desenha_box(frame, boxes, class_ids, confidences):
  for i in range(len(boxes)):
    x,y,w,h,cx,cy = boxes[i]
    label = str(classes[class_ids[i]])
    label += ' '+ str(round(confidences[i],2))
    cor = cores[class_ids[i]]
    cv2.rectangle(frame,(x ,y),(x+w, y+h),cor, 2)
    cv2.putText(frame, label, (x, y+30), cv2.FONT_HERSHEY_PLAIN,1,(255,255,255),2)

In [ ]:
carros_detectados = []

h_bins = 50
s_bins = 60
histSize = [h_bins, s_bins]
# hue varies from 0 to 179, saturation from 0 to 255
h_ranges = [0, 180]
s_ranges = [0, 256]
ranges = h_ranges + s_ranges # concat lists
# Use the 0-th and 1-st channels
channels = [0, 1]

def registrar_carro(frame, box):
  x,y,w,h,cx,cy = box
  crop_box = frame[y:y+h, x:x+w]
  gray_crop_box = cv2.cvtColor(crop_box, cv2.COLOR_BGR2HSV) 
  exists = _is_carro_registrado(gray_crop_box)

  if (not exists) :  
    carros_detectados.append(gray_crop_box)

  return carros_detectados

def _is_carro_registrado(gray_crop_box):
  
  cropped_image_histogram = cv2.calcHist([gray_crop_box], channels, None, histSize, ranges, accumulate=False)

  if (len(carros_detectados) == 0):
    return False
  
  exists = False;

  for detected in carros_detectados:
    detected_hist = cv2.calcHist([detected], channels, None, histSize, ranges, accumulate=False)

    # https://docs.opencv.org/3.4/d8/dc8/tutorial_histogram_comparison.html
    # https://docs.opencv.org/4.1.2/d6/dc7/group__imgproc__hist.html#gaf4190090efa5c47cb367cf97a9a519bd
    # https://docs.opencv.org/4.1.2/d6/dc7/group__imgproc__hist.html#ga994f53817d621e2e4228fc646342d386
    a = cv2.compareHist(cropped_image_histogram, detected_hist, cv2.HISTCMP_CORREL)
    
    if (a > 0.60):
      exists = True
      break 

  return exists

In [ ]:
cap = cv2.VideoCapture('./projeto/videos/Car2165.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
writer = cv2.VideoWriter('./projeto/videos/novo_Car2165.mp4', cv2.VideoWriter_fourcc(*'DIVX'),20,(width, height))

print(width, height)
if cap.isOpened() == False:
    print('error file not found')

cod_frame=0
inicio = int(round(time.time() * 1000))
detectados = []

veiculos = 0

while cap.isOpened():
    cod_frame+=1
    ret, frame = cap.read()
    if ret == True:

        #BGR -> RGB
        blob = cv2.dnn.blobFromImage(frame, 0.00392,(320,320),(0,0,0),True)
        net.setInput(blob)
        outs = net.forward(outputLayers)

        class_ids ,confidences , boxes = encontrar_objetos_dnn(outs)

        desenha_box(frame, boxes, class_ids, confidences)

        for i in range(len(boxes)):
          x,y,w,h,cx,cy = boxes[i]
          veiculos = len(registrar_carro(frame, boxes[i]))
          cv2.circle(frame, (cx,cy) , 4, (0, 0, 255), -1)        

        text = f'veiculos: {veiculos}'

        cv2.putText(frame, text, (0, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 5)      
        
        writer.write(frame)

    else:
        break

cap.release()
writer.release()

1280 720
